In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
# Establish connection
storageAccountName = 'storage1ngest'
storageAccountAccessKey = 'something_goes_here=='
blobContainerName = 'main'


if not any(mount.mountPoint == '/mnt/FileStore/MountFolder/' for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
    source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
    mount_point = "/mnt/FileStore/MountFolder/",
    extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
  )
  except Exception as e:
    print("already mounted. Try to unmount first")

display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/data"))

path,name,size
dbfs:/mnt/FileStore/MountFolder/data/.DS_Store,.DS_Store,6148
dbfs:/mnt/FileStore/MountFolder/data/csv/,csv/,0
dbfs:/mnt/FileStore/MountFolder/data/json/,json/,0


In [ ]:
# Un-Mounter: Connection script
#Only use this to unmount container
#dbutils.fs.unmount("/mnt/mounter1")

/mnt/mounter1 has been unmounted.
Out[4]: True

In [ ]:
# CSV PARSER
# Import Libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType ,LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DateType, TimestampType, DecimalType

import datetime
from typing import List
from decimal import Decimal


# Parse .csv files
def parse_csv(line:str):
    """
      Function to parse comma seperated records
      
    """        
    try:
        # position of the record_type field
        record_type_pos = 2

        # get the common fields applicable to both 'Q' and 'T' type records
        record = line.split(",")
        trade_dt= datetime.datetime.strptime(record[0], '%Y-%m-%d')
        arrival_tm=datetime.datetime.strptime(record[1], '%Y-%m-%d %H:%M:%S.%f')
        rec_type=record[2]
        symbol=record[3]
        exchange=record[6]
        event_tm=datetime.datetime.strptime(record[4], '%Y-%m-%d %H:%M:%S.%f')
        event_seq_nb=int(record[5])
        
        # fields specific to record_type is 'T'
        if record[record_type_pos] == "T":
            trade_pr=Decimal(record[7])
            event = (trade_dt,rec_type,symbol,exchange,event_tm,event_seq_nb,arrival_tm,\
                     trade_pr,Decimal('0.0'),0,Decimal('0.0'),0,"T")
            return event
        
        # fields specific to record_type is 'Q'
        elif record[record_type_pos] == "Q":
            bid_pr=Decimal(record[7])
            bid_size=int(record[8])
            ask_pr=Decimal(record[9])
            ask_size=int(record[10])

            event = (trade_dt,rec_type,symbol,exchange,event_tm,event_seq_nb,arrival_tm,\
                     Decimal('0.0'),bid_pr,bid_size,ask_pr,ask_size,"Q")
            return event
    # capture record in a bad partition if any error occurs
    except Exception as e:
      return ("","","","","","","","","","","","B",line)
        
# schema to parse both Q and T type records
common_event = StructType() \
              .add("trade_dt",DateType(),True) \
              .add("rec_type",StringType(),True) \
              .add("symbol",StringType(),True) \
              .add("exchange",StringType(),True) \
              .add("event_tm",TimestampType(),True) \
              .add("event_seq_nb",IntegerType(),True) \
              .add("arrival_tm",TimestampType(),True) \
              .add("trade_pr",DecimalType(17,14),True) \
              .add("bid_pr",DecimalType(17,14),True) \
              .add("bid_size",IntegerType(),True) \
              .add("ask_pr",DecimalType(17,14),True) \
              .add("ask_size",IntegerType(),True) \
              .add("partition",StringType(),True)
              
## Spark to process the source data (below line is for local execution)
spark = SparkSession.builder.master('local').\
        appName('app').getOrCreate()


raw = spark.sparkContext.\
           textFile("dbfs:/mnt/FileStore/MountFolder/data/csv/*/NYSE/*.txt")

# Parse the text file and parse using the parse_csv function to get the rdd in proper format. 
parsed = raw.map(lambda line: parse_csv(line))
data_csv = spark.createDataFrame(parsed,schema=common_event)

# Save the final dataframe as parquet files in partitions
data_csv.write.partitionBy("partition").mode("overwrite").parquet("dbfs:/mnt/FileStore/MountFolder/output_dir")

In [ ]:
# Display .csv dataframe after parsing
data_csv.show(10, truncate=False)

+----------+--------+------+--------+-----------------------+------------+-------------------+-----------------+-----------------+--------+-----------------+--------+---------+
trade_dt |rec_type|symbol|exchange|event_tm |event_seq_nb|arrival_tm |trade_pr |bid_pr |bid_size|ask_pr |ask_size|partition|
+----------+--------+------+--------+-----------------------+------------+-------------------+-----------------+-----------------+--------+-----------------+--------+---------+
2020-08-05|Q |SYMA |NYSE |2020-08-05 09:34:51.505|1 |2020-08-05 09:30:00|0E-14 |75.30254839137037|100 |75.35916738004924|100 |Q |
2020-08-05|Q |SYMA |NYSE |2020-08-05 09:40:52.586|2 |2020-08-05 09:30:00|0E-14 |77.20874619466693|100 |78.90918015646369|100 |Q |
2020-08-05|Q |SYMA |NYSE |2020-08-05 09:50:04.681|3 |2020-08-05 09:30:00|0E-14 |77.15973273251218|100 |77.33204694110470|100 |Q |
2020-08-05|Q |SYMA |NYSE |2020-08-05 09:57:46.343|4 |2020-08-05 09:30:00|0E-14 |79.29977331004093|100 |80.08399307353596|100 |Q |
2020-08-05|Q |SYMA |NYSE |2020-08-05 10:06:50.886|5 |2020-08-05 09:30:00|0E-14 |77.86349512170780|100 |78.30821537434917|100 |Q |
2020-08-05|Q |SYMA |NYSE |2020-08-05 10:11:43.839|6 |2020-08-05 09:30:00|0E-14 |78.74799689143143|100 |80.32905299746953|100 |Q |
2020-08-05|Q |SYMA |NYSE |2020-08-05 10:17:02.044|7 |2020-08-05 09:30:00|0E-14 |77.16837620945849|100 |77.97598027909252|100 |Q |
2020-08-05|Q |SYMA |NYSE |2020-08-05 10:23:58.216|8 |2020-08-05 09:30:00|0E-14 |78.06551496692607|100 |78.80516026628197|100 |Q |
2020-08-05|Q |SYMA |NYSE |2020-08-05 10:31:40.057|9 |2020-08-05 09:30:00|0E-14 |76.69653634240801|100 |78.56942375735030|100 |Q |
2020-08-05|T |SYMA |NYSE |2020-08-05 10:37:21.581|10 |2020-08-05 09:30:00|79.19488165597565|0E-14 |0 |0E-14 |0 |T |
+----------+--------+------+--------+-----------------------+------------+-------------------+-----------------+-----------------+--------+-----------------+--------+---------+
only showing top 10 rows

In [ ]:
# Save .csv dataframe after parsing as a parquet file
data_csv.write.partitionBy("partition").mode("overwrite").parquet("dbfs:/mnt/FileStore/MountFolder/output_dir")

In [ ]:
# Display status
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/output_dir"))

path,name,size
dbfs:/mnt/FileStore/MountFolder/output_dir/_SUCCESS,_SUCCESS,0
dbfs:/mnt/FileStore/MountFolder/output_dir/_committed_2468780923525334882,_committed_2468780923525334882,35
dbfs:/mnt/FileStore/MountFolder/output_dir/partition=Q/,partition=Q/,0
dbfs:/mnt/FileStore/MountFolder/output_dir/partition=T/,partition=T/,0


In [ ]:
# Display status of Parquet file save
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/output_dir/partition=Q/"))

path,name,size
dbfs:/mnt/FileStore/MountFolder/output_dir/partition=Q/_SUCCESS,_SUCCESS,0
dbfs:/mnt/FileStore/MountFolder/output_dir/partition=Q/_committed_1039567998148144155,_committed_1039567998148144155,222
dbfs:/mnt/FileStore/MountFolder/output_dir/partition=Q/_committed_2468780923525334882,_committed_2468780923525334882,430
dbfs:/mnt/FileStore/MountFolder/output_dir/partition=Q/_started_1039567998148144155,_started_1039567998148144155,0
dbfs:/mnt/FileStore/MountFolder/output_dir/partition=Q/_started_2468780923525334882,_started_2468780923525334882,0
dbfs:/mnt/FileStore/MountFolder/output_dir/partition=Q/part-00000-tid-2468780923525334882-2223570c-e759-4116-9153-e0e14aea4ed2-26-1.c000.snappy.parquet,part-00000-tid-2468780923525334882-2223570c-e759-4116-9153-e0e14aea4ed2-26-1.c000.snappy.parquet,10743
dbfs:/mnt/FileStore/MountFolder/output_dir/partition=Q/part-00001-tid-2468780923525334882-2223570c-e759-4116-9153-e0e14aea4ed2-27-1.c000.snappy.parquet,part-00001-tid-2468780923525334882-2223570c-e759-4116-9153-e0e14aea4ed2-27-1.c000.snappy.parquet,10755


In [ ]:
# JSON PARSER
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType ,LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DateType, TimestampType, DecimalType

import json
from decimal import Decimal 
import datetime

def parse_json(line:str):
    """
      Function to parse json records
      
    """
    try:
        # built-in function to parse the json file
        record = json.loads(line)
        
        # Common fields applicable to both 'Q' and 'T' type records
        record_type = record['event_type']
        trade_dt= datetime.datetime.strptime(record['trade_dt'], '%Y-%m-%d')
        arrival_tm=datetime.datetime.strptime(record['file_tm'], '%Y-%m-%d %H:%M:%S.%f')
        rec_type=record_type
        symbol=record['symbol']
        exchange=record['exchange']
        event_tm=datetime.datetime.strptime(record['event_tm'], '%Y-%m-%d %H:%M:%S.%f')
        event_seq_nb=int(record['event_seq_nb'])

        # capturing field values specific to "T" type records
        if record_type == "T":
            trade_pr=Decimal(record['price'])
            event = (trade_dt,rec_type,symbol,exchange,event_tm,event_seq_nb,arrival_tm,\
                     trade_pr,Decimal('0.0'),0,Decimal('0.0'),0,"T")
            return event
        
        # capturing field values specific to "Q" type records
        elif record_type == "Q":
            bid_pr=Decimal(record['bid_pr'])
            bid_size=int(record['bid_size'])
            ask_pr=Decimal(record['ask_pr'])
            ask_size=int(record['ask_size'])
            event = (trade_dt,rec_type,symbol,exchange,event_tm,event_seq_nb,arrival_tm,\
                     Decimal('0.0'),bid_pr,bid_size,ask_pr,ask_size,"Q")
            return event

    # capture record in a bad partition if any error occurs
    except Exception as e:
            return ("","","","","","","","","","","","B",line)



## Spark to process the source data (below line is for local execution)
spark = SparkSession.builder.master('local').appName('app').getOrCreate()

# schema to parse both Q and T type records
common_event = StructType() \
              .add("trade_dt",DateType(),True) \
              .add("rec_type",StringType(),True) \
              .add("symbol",StringType(),True) \
              .add("exchange",StringType(),True) \
              .add("event_tm",TimestampType(),True) \
              .add("event_seq_nb",IntegerType(),True) \
              .add("arrival_tm",TimestampType(),True) \
              .add("trade_pr",DecimalType(17,14),True) \
              .add("bid_pr",DecimalType(17,14),True) \
              .add("bid_size",IntegerType(),True) \
              .add("ask_pr",DecimalType(17,14),True) \
              .add("ask_size",IntegerType(),True) \
              .add("partition",StringType(),True)


raw = spark.sparkContext.textFile("dbfs:/mnt/FileStore/MountFolder/data/json/*/NASDAQ/*.txt")
parsed = raw.map(lambda line: parse_json(line))
data_json = spark.createDataFrame(parsed,schema=common_event)

# Save the final dataframe as parquet files in partitions
data_json.write.partitionBy("partition").mode("overwrite").parquet("dbfs:/mnt/FileStore/MountFolder/output_dir_json")

In [ ]:
# Display .json file in a dataframe
data_json.show(10,truncate=False)

+----------+--------+------+--------+-----------------------+------------+-------------------+-----------------+-----------------+--------+-----------------+--------+---------+
trade_dt |rec_type|symbol|exchange|event_tm |event_seq_nb|arrival_tm |trade_pr |bid_pr |bid_size|ask_pr |ask_size|partition|
+----------+--------+------+--------+-----------------------+------------+-------------------+-----------------+-----------------+--------+-----------------+--------+---------+
2020-08-05|Q |SYMA |NASDAQ |2020-08-05 09:36:55.284|1 |2020-08-05 09:30:00|0E-14 |76.10016521142818|100 |77.96479759087470|100 |Q |
2020-08-05|Q |SYMA |NASDAQ |2020-08-05 09:42:32.247|2 |2020-08-05 09:30:00|0E-14 |75.44372945251948|100 |75.94452858561046|100 |Q |
2020-08-05|Q |SYMA |NASDAQ |2020-08-05 09:48:06.767|3 |2020-08-05 09:30:00|0E-14 |78.84798564828422|100 |80.69114407667608|100 |Q |
2020-08-05|Q |SYMA |NASDAQ |2020-08-05 09:53:09.803|4 |2020-08-05 09:30:00|0E-14 |74.98336890552693|100 |76.16256530811053|100 |Q |
2020-08-05|Q |SYMA |NASDAQ |2020-08-05 10:00:10.866|5 |2020-08-05 09:30:00|0E-14 |76.71408448666702|100 |77.85512785142082|100 |Q |
2020-08-05|Q |SYMA |NASDAQ |2020-08-05 10:05:24.344|6 |2020-08-05 09:30:00|0E-14 |76.78900747031288|100 |77.79982571852163|100 |Q |
2020-08-05|Q |SYMA |NASDAQ |2020-08-05 10:13:59.224|7 |2020-08-05 09:30:00|0E-14 |76.37099394549293|100 |76.94631910510188|100 |Q |
2020-08-05|Q |SYMA |NASDAQ |2020-08-05 10:21:43.76 |8 |2020-08-05 09:30:00|0E-14 |77.32601214334609|100 |77.65017757988643|100 |Q |
2020-08-05|Q |SYMA |NASDAQ |2020-08-05 10:30:33.547|9 |2020-08-05 09:30:00|0E-14 |77.11395059665263|100 |77.52985832434904|100 |Q |
2020-08-05|T |SYMA |NASDAQ |2020-08-05 10:38:50.046|10 |2020-08-05 09:30:00|77.77570455205036|0E-14 |0 |0E-14 |0 |T |
+----------+--------+------+--------+-----------------------+------------+-------------------+-----------------+-----------------+--------+-----------------+--------+---------+
only showing top 10 rows

In [ ]:
# Save .json dataframe after parsing as a parquet file
data_json.write.partitionBy("partition").mode("overwrite").parquet("dbfs:/mnt/FileStore/MountFolder/output_dir_json")

In [ ]:
# Display status of .json file write
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/output_dir_json"))

path,name,size
dbfs:/mnt/FileStore/MountFolder/output_dir_json/_SUCCESS,_SUCCESS,0
dbfs:/mnt/FileStore/MountFolder/output_dir_json/_committed_8671808673027994356,_committed_8671808673027994356,35
dbfs:/mnt/FileStore/MountFolder/output_dir_json/partition=Q/,partition=Q/,0
dbfs:/mnt/FileStore/MountFolder/output_dir_json/partition=T/,partition=T/,0


In [ ]:
# Display status of Parquet file save
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/output_dir_json/partition=Q/"))

path,name,size
dbfs:/mnt/FileStore/MountFolder/output_dir_json/partition=Q/_SUCCESS,_SUCCESS,0
dbfs:/mnt/FileStore/MountFolder/output_dir_json/partition=Q/_committed_2265823041017308852,_committed_2265823041017308852,222
dbfs:/mnt/FileStore/MountFolder/output_dir_json/partition=Q/_committed_8671808673027994356,_committed_8671808673027994356,430
dbfs:/mnt/FileStore/MountFolder/output_dir_json/partition=Q/_started_2265823041017308852,_started_2265823041017308852,0
dbfs:/mnt/FileStore/MountFolder/output_dir_json/partition=Q/_started_8671808673027994356,_started_8671808673027994356,0
dbfs:/mnt/FileStore/MountFolder/output_dir_json/partition=Q/part-00000-tid-8671808673027994356-45bf1b3f-dec0-4a39-a27e-7f967bff8097-37-1.c000.snappy.parquet,part-00000-tid-8671808673027994356-45bf1b3f-dec0-4a39-a27e-7f967bff8097-37-1.c000.snappy.parquet,10764
dbfs:/mnt/FileStore/MountFolder/output_dir_json/partition=Q/part-00001-tid-8671808673027994356-45bf1b3f-dec0-4a39-a27e-7f967bff8097-38-1.c000.snappy.parquet,part-00001-tid-8671808673027994356-45bf1b3f-dec0-4a39-a27e-7f967bff8097-38-1.c000.snappy.parquet,10772
